In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
# cd '../gdrive/MyDrive/3조'
# cd '../gdrive/MyDrive/SSAC/3조'

In [ ]:
!pip uninstall lightgbm -y
!pip install lightgbm --install-option=--gpu

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime, date, time
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =999
pd.options.display.max_rows = 999

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("data/merged_data_notext.csv")

In [ ]:
# 클릭율
grouped_label = df.groupby('label').size()
average_ctr = float(grouped_label[1]/grouped_label.sum())
average_ctr

0.03200107653453746

# 평가지표 함수

In [ ]:
# 평가지표 함수
def get_rig(train_y, test_y, pred):
    avg_ctr = average_ctr
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig

# 학습과정 전처리 함수

In [ ]:
# 전처리 함수
def process_missing_values(df):
    for categorical_col in categorical:
        df[categorical_col] = df[categorical_col].astype(str)
        df[categorical_col] = df[categorical_col].fillna('0')
        df[categorical_col] = preprocessing.LabelEncoder().fit_transform(df[categorical_col])

    for continuous_col in continuous:
        df[continuous_col] = df[continuous_col].fillna(0)

    return df

# LGB 모델 

In [ ]:
# Train_test_split
def split_dataset(df, features):
    train_test_df = df[['label'] + features]
    train, test = train_test_split(train_test_df, test_size = 0.2, random_state=47)

    X_train = train[features]
    y_train = train['label']

    X_test = test[features]
    y_test = test['label']
    return X_train, y_train, X_test, y_test

In [ ]:
# 학습
def train_lgb(X_train, y_train):
    model = lgb.LGBMClassifier(n_estimators=50,
        learning_rate=0.1,
        num_leaves=127,
        max_depth=15,
        zero_as_missing=True,
        n_jobs=os.cpu_count(),
        objective='binary')

    print('start training')
    model.fit(X_train, y_train)
    return model
    
# 예측
def evaluate_lgb(model, X_test):
    print('predicting')
    pred = model.predict_proba(X_test)[:,1]

    print(f'auc : {roc_auc_score(y_test, pred)}, rig: {get_rig(y_train, y_test, pred)}')

# 학습 피쳐 조정

In [ ]:
categorical = [
    'viewer_gender',  
    'content_used',
    'content_cat_1',
    'content_cat_2',
    'content_cat_3',
    "content_status"]

continuous = [
    'bid_price', 
    'content_price',
    'content_emergency_count',
    'content_comment_count',     
    'content_views',       
    'content_likes',    
    'adv_follower_count',
    'adv_grade',       
    'adv_item_count',         
    'adv_views', # adv_reivew drop
    'adv_review_count',
    'adv_comment_count',
    'adv_pay_count',
    'adv_parcel_post_count', 
    'adv_transfer_count', 
    'adv_chat_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    #"viewer_trans_pay_count", # viewer_trans drop
    'viewer_transfer_count',
    'viewer_chat_count']

features = categorical + continuous
df = process_missing_values(df)

In [ ]:
# features만 수정하면 됨
X_train, y_train, X_test, y_test = split_dataset(df, features)

In [ ]:
model = train_lgb(X_train, y_train)

start training


In [ ]:
evaluate_lgb(model, X_test)

predicting
auc : 0.744958366806391, rig: 0.1010414869480056
